# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
## Учимся переводить между decision/evaluation/search на примере Bin Packing

**Даны:** 
* $[w_1, \dots , w_k], w_i \in [0, 1]$ — веса грузов  
* $n_{\text{bins}}$ — количество корзин грузоподъемностью 1.

Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? 

Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing). 

Затем напишите содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). 

Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [47]:
def solve_bp_decision(weights: list, n_bins: int) -> bool:
    eps = 1e-7
    def able_to_pack(weights: list, bin_capacities) -> bool:
        return weights == [] or any( 
            able_to_pack( weights[:-1], bin_capacities[:i] + [capacity - weights[-1]] + bin_capacities[(i + 1):] ) 
            for i, capacity in enumerate(bin_capacities) if (capacity > weights[-1] + eps or abs(capacity - weights[-1]) < eps)
        )

    return able_to_pack( sorted(weights), [1.0] * n_bins )

# To test this, run:
# solve_bp_decision([0.8, 0.09, 0.4, 0.7], 2)
# solve_bp_decision([0.8, 0.09, 0.4, 0.7], 3)

In [46]:
def solve_bp_evaluation(weights: list) -> int:
    l = 0
    r = len(weights)
    
    while (l != r):
        m = (l + r) // 2
        
        if (solve_bp_decision(weights, m)):
            r = m
        else:
            l = m + 1
    
    return l
    
    pass

In [49]:
def check(weights, c):
    eps = 1e-7
    w = []
    
    for i in range(len(c)):
        now = int(0)
        
        for j in range(len(c[i])):
            now += weights[c[i][j]]
        
        if (now > 1 + eps):
            return False
        
        w.append(now)
    
    return solve_bp_evaluation(w)

    pass

def solve_bp_search(weights: list):
    if (len(weights) == 0):
        return []
    
    ans = solve_bp_evaluation(weights)
    c = []
    
    for i in range(len(weights)):
        c.append([i])
    
    while (len(c) != ans):
        st = False
        for i in range(len(c)):
            for j in range(i + 1, len(c)):
                if (check(weights, c[:i] + [c[i] + c[j]] + c[i + 1:j] + c[j + 1:len(c)]) == ans):
                    c = c[:i] + [c[i] + c[j]] + c[i + 1:j] + c[j + 1:len(c)]
                    st = True
                    break
            
            if (st):
                break
    
    ans = [0] * len(weights)
    
    for i in range(len(c)):
        for j in range(len(c[i])):
            ans[c[i][j]] = i + 1
            
    return ans
    
    pass

#solve_bp_search([0.65, 0.3, 0.1, 0.05, 0.9])